In [37]:
import sklearn

print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.19.2.


In [40]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-

import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing

#random_state = 42

########################################
# LOADING DATASETS IN PANDA DATAFRAMES #
########################################

train_csv_path = "./all/train.csv"
test_csv_path = "./all/test.csv"

train_df = pd.read_csv(train_csv_path,header=0)
test_df = pd.read_csv(test_csv_path,header=0)

# split df so that ids are isolated from features
# on drop la premiere colonne d'id, qui n'est pas un feature, cf. exemple tuto xgboost
train_data, train_target = train_df.iloc[:,1:-1], train_df.iloc[:,-1]
# pour TEST pas de classe (test_target) !!!: 
# on fait juste une prédiction de probabilité qu'on envoie sur Kaggle
test_data = test_df.iloc[:,1:]

# transformer les string de classe en etiquette int pour train
#nbrRows = train_target.shape[0]
#for rowIndex in range(nbrRows):
#    train_target.at[rowIndex] = int(train_target.at[rowIndex][6])

#pd.to_numeric(train_target)

In [41]:
print(train_data.head())
print(train_target.head())
print(train_data.values.shape)
print(type(train_data.values[0][0]))
print(train_target.values.shape)
print(type(train_target.values[0]))

   feat_1  feat_2  feat_3  feat_4  feat_5  feat_6  feat_7  feat_8  feat_9  \
0       1       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       1       0   
2       0       0       0       0       0       0       0       1       0   
3       1       0       0       1       6       1       5       0       0   
4       0       0       0       0       0       0       0       0       0   

   feat_10   ...     feat_84  feat_85  feat_86  feat_87  feat_88  feat_89  \
0        0   ...           0        1        0        0        0        0   
1        0   ...           0        0        0        0        0        0   
2        0   ...           0        0        0        0        0        0   
3        1   ...          22        0        1        2        0        0   
4        0   ...           0        1        0        0        0        0   

   feat_90  feat_91  feat_92  feat_93  
0        0        0        0      

# À propos des paramètres de GridSearchCV

cf. http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
conseils sur les valeurs de paramètres:
https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/


## REFIT

refit : boolean, or string, default=True
Refit an estimator using the best found parameters on the whole dataset.

For multiple metric evaluation, this needs to be a string denoting the scorer is used to find the best parameters for refitting the estimator at the end.

The refitted estimator is made available at the best_estimator_ attribute and permits using predict directly on this GridSearchCV instance.

Also for multiple metric evaluation, the attributes best_index_, best_score_ and best_params_ will only be available if refit is set and all of them will be determined w.r.t this specific scorer.

## SCORING

http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

## Arguments du .fit

X : array-like, shape = [n_samples, n_features]
Training vector, where n_samples is the number of samples and n_features is the number of features.

y : array-like, shape = [n_samples] or [n_samples, n_output], optional
Target relative to X for classification or regression; None for unsupervised learning
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.values.html#pandas.DataFrame.values



In [51]:
########################################################
# GridSearchCV on part of the possible hyperparameters #
########################################################

xgb_classifier = xgb.XGBClassifier()

parameters = {'objective':['binary:logistic'],             
              'colsample_bytree':[0.4,0.5,0.7],
              'learning_rate':[0.1],
              'max_depth':[5,6],
              'reg_alpha':[0,1,2],
              'n_estimators':[100],
              'seed':[42],
              'nthread':[3]}

classifier = GridSearchCV(xgb_classifier,parameters,verbose=2,scoring=["f1_micro"],refit="f1_micro")
classifier.fit(train_data.values,train_target.values)



Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] colsample_bytree=0.4, learning_rate=0.1, max_depth=5, n_estimators=100, nthread=3, objective=binary:logistic, reg_alpha=0, seed=42 


/home/mbauw/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mbauw/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   23.5s remaining:    0.0s


[CV]  colsample_bytree=0.4, learning_rate=0.1, max_depth=5, n_estimators=100, nthread=3, objective=binary:logistic, reg_alpha=0, seed=42, total=  22.5s
[CV] colsample_bytree=0.4, learning_rate=0.1, max_depth=5, n_estimators=100, nthread=3, objective=binary:logistic, reg_alpha=0, seed=42 


/home/mbauw/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mbauw/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  colsample_bytree=0.4, learning_rate=0.1, max_depth=5, n_estimators=100, nthread=3, objective=binary:logistic, reg_alpha=0, seed=42, total=  22.8s
[CV] colsample_bytree=0.4, learning_rate=0.1, max_depth=5, n_estimators=100, nthread=3, objective=binary:logistic, reg_alpha=0, seed=42 


...

/home/mbauw/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mbauw/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  colsample_bytree=0.7, learning_rate=0.1, max_depth=6, n_estimators=100, nthread=3, objective=binary:logistic, reg_alpha=2, seed=42, total=  40.7s
[CV] colsample_bytree=0.7, learning_rate=0.1, max_depth=6, n_estimators=100, nthread=3, objective=binary:logistic, reg_alpha=2, seed=42 


/home/mbauw/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mbauw/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed: 28.8min finished


[CV]  colsample_bytree=0.7, learning_rate=0.1, max_depth=6, n_estimators=100, nthread=3, objective=binary:logistic, reg_alpha=2, seed=42, total=  40.7s


GridSearchCV(cv=None, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'objective': ['binary:logistic'], 'colsample_bytree': [0.4, 0.5, 0.7], 'learning_rate': [0.1], 'max_depth': [5, 6], 'reg_alpha': [0, 1, 2], 'n_estimators': [100], 'seed': [42], 'nthread': [3]},
       pre_dispatch='2*n_jobs', refit='f1_micro',
       return_train_score='warn', scoring=['f1_micro'], verbose=2)

In [52]:
################################
# Show results of GridSearchCV #
################################

print(classifier.cv_results_)
classifier.best_params_

{'mean_fit_time': array([22.82486884, 25.25702802, 21.84668501, 26.66145047, 26.38067142,
       26.31171902, 25.56809251, 25.39713907, 27.50464288, 33.45486204,
       31.5140202 , 30.65272657, 33.68358548, 33.46049968, 33.01726683,
       40.16932336, 40.02784173, 39.97566835]), 'std_fit_time': array([1.04233304, 3.75990061, 0.13604785, 0.04794223, 0.05557851,
       0.10423215, 0.15755359, 0.04804934, 2.64788352, 0.69974269,
       0.2538939 , 0.16149946, 0.78994813, 0.50224678, 0.13043733,
       0.13471566, 0.1172104 , 0.18379597]), 'mean_score_time': array([0.55579813, 0.71059942, 0.50417296, 0.64239542, 0.60464883,
       0.60298975, 0.51543554, 0.51468158, 0.55750124, 0.63731345,
       0.6116124 , 0.59281452, 0.50583839, 0.49284601, 0.48734021,
       0.62815515, 0.58464344, 0.58658536]), 'std_score_time': array([0.03528071, 0.25056108, 0.00991988, 0.0126386 , 0.00199198,
       0.00876507, 0.00639322, 0.0095465 , 0.0529283 , 0.01283534,
       0.00253615, 0.00749034, 0.011447

{'colsample_bytree': 0.7,
 'learning_rate': 0.1,
 'max_depth': 6,
 'n_estimators': 100,
 'nthread': 3,
 'objective': 'binary:logistic',
 'reg_alpha': 1,
 'seed': 42}

# Now run with best parameters found thanks to GridSearchCV

In [53]:
classifier.best_params_['seed']

# regarding python kwards to forward best parameters check url below
# https://stackoverflow.com/questions/41475539/using-best-params-from-gridsearchcv
xgb_best_classifier = xgb.XGBClassifier(**classifier.best_params_)

xgb_best_classifier.fit(train_data,train_target)
preds = xgb_best_classifier.predict_proba(test_data)

In [54]:
#####################################################
# CREATING .CSV RESPECTING KAGGLE SUBMISSION FORMAT #
#####################################################

preds = pd.DataFrame(preds)
namesRow = ["Class_1","Class_2","Class_3","Class_4","Class_5","Class_6","Class_7","Class_8","Class_9"]
preds.columns = namesRow
preds.head()

preds.index +=1
preds.to_csv("results_grid_xgboost.csv", encoding='utf-8',index=True,index_label="id")